In [2]:
import ee
import pandas as pd
import os
import xarray as xr

In [ ]:
ee.Authenticate()

In [3]:
ee.Initialize()

In [22]:
directory = "/home/khanalp/task1/data/ICOS/Input_data"
nc_files = [file for file in os.listdir(directory) if file.endswith(".nc")]

In [23]:
# This contains all the stations of the ICOS with information like plant canopy height, measurement height, etc. 
station_all = pd.read_excel("station_with_elevation_heightcanopy.xlsx")
# Set the "station_name" column as the index
station_all = station_all.set_index('station_name')

In [24]:
# selected_stations contains the stations with no NA values"
selected_station = pd.read_csv("sites_with_noNAs.csv", index_col= 0)

# These are station for which we will drive the model. 
station = station_all[station_all.index.isin(selected_station.index)]

In [25]:
# Initialize an empty dictionary to store station_name, latitude, and longitude and other information..
station_dict = {}
# Iterate over each row in the DataFrame
for index, row in station.iterrows():
    # Extract the station_name and position
    station_name = index
    latitude = row['latitude']
    longitude = row['longitude']
    elevation = row['elevation']
    IGBP_longname = row['IGBP long name']
    IGBP_shortname = row['IGBP short name']
    #start_year = row['Start_Year_Threshold']
    #end_year = row['End_Year_Threshold']
    
    # Extract height_canopy from 'height_canopy_field_information' column or 'height_canopy_ETH' column
    height_canopy = row['height_canopy_field_information']
    if pd.isna(height_canopy):  # Check if height_canopy_field_information is NaN
        height_canopy = row['height_canopy_ETH']

    measurement_height = row['Measurement height']
    
    # Store the station_name, latitude, and longitude in the dictionary
    station_dict[station_name] = {
        'latitude': latitude, 
        'longitude': longitude, 
        'elevation' : elevation,
        'IGBP_longname': IGBP_longname,
        'IGBP_shortname': IGBP_shortname,
        'height_canopy': height_canopy,
        'measurement_height': measurement_height,
    }

In [26]:
# Initialize a dictionary to store station information
station_info = {}

# Iterate over the NetCDF files
for nc_file in nc_files:
    # Extract station name from the filename
    station_name = nc_file.split("_")[0]
    
    # Extract start and end years from the filename
    start_year = int(nc_file.split("_")[1][:4])
    end_year = int(nc_file.split("_")[1][5:9])
    
    # Update station_info dictionary
    station_info[station_name] = {'start_year': start_year, 'end_year': end_year}

In [27]:
# Iterate over station_dict and update each station's dictionary
for station_name, info in station_dict.items():
    # Check if the station_name exists in station_info
    if station_name in station_info:
        # Update the station's dictionary with additional keys from station_info
        additional_info = station_info[station_name]
        info.update(additional_info)

In [28]:
band_required = ['skin_temperature',
                 'soil_temperature_level_1',
                 'soil_temperature_level_2',
                 'soil_temperature_level_3',
                 'soil_temperature_level_4',
                 'volumetric_soil_water_layer_1',
                 'volumetric_soil_water_layer_2',
                 'volumetric_soil_water_layer_3',
                 'volumetric_soil_water_layer_4']

In [29]:
# Loop through each station in station_dict
for station_name, station_info in station_dict.items():
    # Extract latitude, longitude, start_year, and end_year for the current station
    station_id = station_name
    latitude = station_info['latitude']
    longitude = station_info['longitude']
    start_year = station_info['start_year']
    end_year = station_info['end_year']
    break

In [30]:
# Create a point geometry using the latitude and longitude
point = ee.Geometry.Point(longitude, latitude)

In [31]:
# Filter the image collection to include only the images for the specified start year
filtered_collection = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate(str(start_year) + "-01-01", str(start_year) + "-12-31")

# Get the first image from the filtered collection
first_image = ee.Image(filtered_collection.first())

In [32]:
# Extract pixel values at the point for the first image of the year
values = first_image.select(band_required).reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=1000)

In [33]:
# Create a Python dictionary containing the values for the specified bands
# Extract values for the specified bands at the given location
values_dict = first_image.reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=10000).getInfo()

# Create a pandas DataFrame from the dictionary of band values
df = pd.DataFrame(values_dict, index=[0])

# Select only the required bands
df = df[band_required]

df.index = [station_name]

In [34]:
# Create an empty list to store DataFrames for each station
dfs = []

# Loop through each station in station_dict
for station_name, station_info in station_dict.items():
    # Extract latitude, longitude, start_year, and end_year for the current station
    latitude = station_info['latitude']
    longitude = station_info['longitude']
    start_year = station_info['start_year']
    end_year = station_info['end_year']
    
    # Create a point geometry using the latitude and longitude
    point = ee.Geometry.Point(longitude, latitude)

    
    # Filter the image collection to include only the images for the specified start year
    filtered_collection = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate(str(start_year) + "-01-01", str(start_year) + "-12-31")

    # Get the first image from the filtered collection
    first_image = ee.Image(filtered_collection.first())
    
    # Get the date of the first image
    image_date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    
    # Extract pixel values at the point for the first image of the year
    values_dict = first_image.reduceRegion(reducer=ee.Reducer.first(), geometry=point, scale=10000).getInfo()
    
    # Create a pandas DataFrame from the dictionary of band values
    station_df = pd.DataFrame(values_dict, index=[station_name])
    
    # Select only the required bands
    station_df = station_df[band_required]

    # Add latitude, longitude, and image date as columns
    station_df['latitude'] = latitude
    station_df['longitude'] = longitude
    station_df['image_date'] = image_date
    
    # Append the station DataFrame to the list
    dfs.append(station_df)

# Concatenate all DataFrames in the list into a single DataFrame
df = pd.concat(dfs)

In [35]:
df

,skin_temperature,soil_temperature_level_1,soil_temperature_level_2,soil_temperature_level_3,soil_temperature_level_4,volumetric_soil_water_layer_1,volumetric_soil_water_layer_2,volumetric_soil_water_layer_3,volumetric_soil_water_layer_4,latitude,longitude,image_date
BE-Lon,283.001587,281.349380,280.044296,280.035721,283.202637,0.410507,0.413544,0.396851,0.343674,50.551620,4.746234,2012-01-01
CZ-KrP,274.378357,273.626114,274.277328,277.239838,280.646927,0.345505,0.351242,0.364731,0.365387,49.573257,15.078773,2014-01-01
FI-Qvd,273.757843,274.273071,274.815628,276.230698,278.926910,0.308960,0.320221,0.326996,0.296204,60.295242,22.391607,2018-01-01
FR-Aur,283.609009,282.904068,282.298203,282.639236,288.144043,0.353378,0.336655,0.260178,0.274643,43.549650,1.106103,2012-01-01
FR-Gri,284.257446,283.060318,282.048203,281.248611,284.985840,0.395081,0.393890,0.352097,0.302551,48.844220,1.951910,2012-01-01
DE-RuS,282.357056,280.620865,279.247421,279.744705,283.062012,0.404297,0.401428,0.359100,0.328949,50.865906,6.447145,2012-01-01
IT-BCi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.523750,14.957444,2012-01-01
DK-Sor,275.067993,275.363052,275.825546,278.856033,281.813965,0.371368,0.376480,0.379166,0.360229,55.485870,11.644645,2012-01-01
FR-Fon,284.327759,283.073990,282.007187,281.109940,285.231934,0.384308,0.379318,0.313004,0.285812,48.476357,2.780096,2012-01-01
FR-Hes,276.278748,276.428848,276.798813,279.274994,281.965286,0.391190,0.398315,0.405212,0.395447,48.674100,7.064650,2014-01-01


In [36]:
df.to_csv("ERA5LandInitialcondition.csv", index = True)